### Making Data Set

In [ ]:
import cv2
import csv
from mediapipe.python.solutions import drawing_utils as mp_drawing, pose as mp_pose

pose_list = ['stop', 'forward', 'backward', 'left', 'right']

# cap = cv2.VideoCapture(f'video\\{pos_name}.mp4') # this is for video
# cap = cv2.VideoCapture(0)  # this is for camera

MY_POSE_CONNECTIONS = frozenset([(16, 14), (14, 12), (12, 11), (11,13), (13,15)])


def writeCSV(pos: str, landmarks):
	with open(f'output_csv\\all.csv', 'a', newline='') as csvfile:
		fieldnames = [
			'POSE',
			'R_SHOULDER_X', 'R_SHOULDER_Y', 'R_SHOULDER_Z', 'R_SHOULDER_V',
			'R_ELBOW_X', 'R_ELBOW_Y', 'R_ELBOW_Z', 'R_ELBOW_V',
			'R_WRIST_X','R_WRIST_Y', 'R_WRIST_Z', 'R_WRIST_V',
			'L_SHOULDER_X', 'L_SHOULDER_Y', 'L_SHOULDER_Z', 'L_SHOULDER_V',
			'L_ELBOW_X', 'L_ELBOW_Y', 'L_ELBOW_Z', 'L_ELBOW_V',
			'L_WRIST_X', 'L_WRIST_Y', 'L_WRIST_Z', 'L_WRIST_V']
		writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

		if csvfile.tell() == 0:
			writer.writeheader()

		R_SHOULDER = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
		R_ELBOW = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
		R_WRIST = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
		L_SHOULDER = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
		L_ELBOW = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
		L_WRIST = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

		data = {
		'POSE': pos,

		'R_SHOULDER_X': R_SHOULDER.x,
		'R_SHOULDER_Y': R_SHOULDER.y,
		'R_SHOULDER_Z': R_SHOULDER.z,
		'R_SHOULDER_V': R_SHOULDER.visibility,

		'R_ELBOW_X': R_ELBOW.x,
		'R_ELBOW_Y': R_ELBOW.y,
		'R_ELBOW_Z': R_ELBOW.z,
		'R_ELBOW_V': R_ELBOW.visibility,

		'R_WRIST_X': R_WRIST.x,
		'R_WRIST_Y': R_WRIST.y,
		'R_WRIST_Z': R_WRIST.z,
		'R_WRIST_V': R_WRIST.visibility,

		'L_SHOULDER_X': L_SHOULDER.x,
		'L_SHOULDER_Y': L_SHOULDER.y,
		'L_SHOULDER_Z': L_SHOULDER.z,
		'L_SHOULDER_V': L_SHOULDER.visibility,

		'L_ELBOW_X': L_ELBOW.x,
		'L_ELBOW_Y': L_ELBOW.y,
		'L_ELBOW_Z': L_ELBOW.z,
		'L_ELBOW_V': L_ELBOW.visibility,

		'L_WRIST_X': L_WRIST.x,
		'L_WRIST_Y': L_WRIST.y,
		'L_WRIST_Z': L_WRIST.z,
		'L_WRIST_V': L_WRIST.visibility
	}

		print(data)
		writer.writerow(data)

def writeDataSet(pos_name: str):
	cap = cv2.VideoCapture(f'video\\{pos_name}.mp4')
	with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
		while cap.isOpened():
			ret, img = cap.read()

			if (ret != True):
				break

			img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
			img.flags.writeable = False

			# Body Detection
			results = pose.process(img)

			img.flags.writeable = True
			img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

			if (results.pose_landmarks):
				mp_drawing.draw_landmarks(img, results.pose_landmarks, MY_POSE_CONNECTIONS)
				writeCSV(pos_name, results.pose_landmarks.landmark)

			cv2.imshow('raw cam', img)
			if cv2.waitKey(1) & 0xFF == ord('e'):
				break

		cap.release()
		cv2.destroyAllWindows()

for pose in pose_list:
	writeDataSet(pose)

### Training and save model

In [9]:
import csv
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

x_data = []
y_data = []

pose_list = ['stop', 'forward', 'backward', 'left', 'right']

with open('output_csv\\all.csv', mode ='r')as file:
	csvFile = csv.DictReader(file)
	# print(csvFile.fieldnames)
	for line in csvFile:
		y_data.append(pose_list.index(line['POSE']))
		x_data.append([float(line[key]) for key in line if key not in ['POSE']])

x_data = np.array(x_data)
y_data = np.array(y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=42)

model = tf.keras.models.Sequential([
	tf.keras.layers.Flatten(input_shape=[24, ]),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(5, activation='softmax'),
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=16)

model.evaluate(x_test, y_test)
model.save('pose.keras')

Epoch 1/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6078 - loss: 1.2060
Epoch 2/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8827 - loss: 0.4425
Epoch 3/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9101 - loss: 0.2913
Epoch 4/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9214 - loss: 0.2485
Epoch 5/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9408 - loss: 0.2073
Epoch 6/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9466 - loss: 0.1777
Epoch 7/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9450 - loss: 0.1653
Epoch 8/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9567 - loss: 0.1391
Epoch 9/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9643 - loss: 0.1202
Epoch 10/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9637 - loss: 0.1217
Epoch 11/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9678 - loss: 0.1000
Epoch 12/16
184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [10]:
import numpy as np

pose_list = ['stop', 'forward', 'backward', 'left', 'right']
pose_list[np.argmax(model.predict(np.array([x_data[2848]])))]

# np.array([x_data[2848]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


'backward'

In [12]:
import cv2, numpy as np, tensorflow as tf
from mediapipe.python.solutions import drawing_utils as mp_drawing, pose as mp_pose

_model = tf.keras.models.load_model('pose.keras')

pose_list = ['stop', 'forward', 'backward', 'left', 'right']
default_pose = pose_list[0]

cap = cv2.VideoCapture(0)  # this is for camera

MY_POSE_CONNECTIONS = frozenset([(16, 14), (14, 12), (12, 11), (11,13), (13,15)])

def getValueFromIMP(landmarks):
		R_SHOULDER = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
		R_ELBOW = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
		R_WRIST = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
		L_SHOULDER = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
		L_ELBOW = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
		L_WRIST = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

		return np.array([[
			R_SHOULDER.x, R_SHOULDER.y, R_SHOULDER.z, R_SHOULDER.visibility,
			R_ELBOW.x, R_ELBOW.y, R_ELBOW.z, R_ELBOW.visibility,
			R_WRIST.x, R_WRIST.y, R_WRIST.z, R_WRIST.visibility,
			L_SHOULDER.x, L_SHOULDER.y, L_SHOULDER.z, L_SHOULDER.visibility,
			L_ELBOW.x, L_ELBOW.y, L_ELBOW.z, L_ELBOW.visibility,
			L_WRIST.x, L_WRIST.y, L_WRIST.z, L_WRIST.visibility
		]])

def getPose(arr):
	pose_list = ['stop', 'forward', 'backward', 'left', 'right']
	return(pose_list[np.argmax(arr)])

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
	while cap.isOpened():
		ret, img = cap.read()

		if (ret != True):
			break

		img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
		img.flags.writeable = False

		# Body Detection
		results = pose.process(img)

		img.flags.writeable = True
		img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

		if (results.pose_landmarks):
			mp_drawing.draw_landmarks(img, results.pose_landmarks, MY_POSE_CONNECTIONS)
			np_output = _model.predict(getValueFromIMP(results.pose_landmarks.landmark))
			current_pose = getPose(np_output)
		else:
			current_pose = default_pose

		cv2.putText(img, current_pose, (50, 50), 1, 2, (0, 0, 255), 3, cv2.FONT_HERSHEY_SIMPLEX)

		cv2.imshow('raw cam', img)

		if cv2.waitKey(1) & 0xFF == ord('e'):
			break

	cap.release()
	cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━